<div style="
  padding: 18px 24px;
  margin-top: 15px;
  margin-bottom: 25px;
  border-left: 6px solid var(--jp-info-color0);
  background: var(--jp-layout-color2);
  border-radius: 8px;
">
<h1 style="margin:0; color:var(--jp-ui-font-color1);">
05. Feature Engineering
</h1>
<p style="font-size: 1.1em; color: var(--jp-ui-font-color1); margin-top: 6px;">
В этом ноутбуке мы описываем и фиксируем, какие признаки (features) используем и как их преобразуем перед обучением модели.
</p>
</div>


<div style="
  border-left: 4px solid #3a3a3a;
  background: var(--jp-layout-color2);
  padding: 12px 14px;
  border-radius: 6px;
  margin: 12px 0;
  color: var(--jp-ui-font-color1);
">
<strong>ℹ️ Информация:</strong> 
 
Таргет: **NObeyesdad_norm** — укрупнённые 4 класса (Insufficient_Weight / Normal_Weight / Overweight / Obesity). 

Будем сравнить два набора признаков:
  - **with_body**: используем Height, Weight, BMI
  - **no_body**: исключаем Height, Weight, BMI
</div>


<h2 style="
  padding: 6px 0;
  border-bottom: 2px solid var(--jp-border-color2);
  /* color: var(--jp-ui-font-color1); */
  margin-top: 32px;
">
05.1 Сформируем наборы признаков
</h2>

In [1]:
# --- Импорты ---

from pathlib import Path
import sys

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from utils import (
    load_clean_df,
    load_columns_mapping,
    cm_columns,
    cm_group,
    cm_group_rank,
    cm_label,
    cm_target_categories,
    cm_target_category_label,
    cm_target_order,
)


In [3]:
# Загрузим очищенный датасет
df = load_clean_df(column_names=None)

# Отобразим типы колонок
display(df.dtypes)


Gender                            category
Age                                float64
Height                             float64
Weight                             float64
family_history_with_overweight    category
FAVC                              category
FCVC                              category
NCP                               category
CAEC                              category
SMOKE                             category
CH2O                              category
SCC                               category
FAF                               category
TUE                               category
CALC                              category
MTRANS                            category
NObeyesdad                        category
NObeyesdad_norm                   category
BMI                                float64
dtype: object

In [4]:
# Выделим таргет тест

TARGET_COL = 'NObeyesdad_norm'
FOR_DROP_COL = ['NObeyesdad', 'BMI']

y = df[TARGET_COL]

cm = load_columns_mapping()
categories = cm_target_categories(TARGET_COL)
order = cm_target_order(TARGET_COL) or []
ru_map = {cls: cm_target_category_label(TARGET_COL, cls) for cls in categories.keys()}

print('Таргет:')
y_info_df = y.value_counts(dropna=False).rename("count").reset_index()
y_info_df["description_ru"] = y_info_df[TARGET_COL].map(ru_map)
if order:
    y_info_df = y_info_df.set_index(TARGET_COL).reindex(order).reset_index()
display(y_info_df)


Таргет:


,NObeyesdad_norm,count,description_ru
0,Insufficient_Weight,267,Недостаточный вес
1,Normal_Weight,282,Норма
2,Overweight,566,Избыточный вес
3,Obesity,972,Ожирение


In [5]:
BODY_COLS = ['Height', 'Weight']

# Все признаки — это всё, кроме таргета
X_all = df.drop(columns=[TARGET_COL] + FOR_DROP_COL, errors='ignore')

# Набор #1: с антропометрией
X_with_body = X_all.copy()

# Набор #2: без антропометрии
X_no_body = X_all.drop(columns=BODY_COLS, errors='ignore')

cm = load_columns_mapping()
group_rank = cm_group_rank()

def make_features_info_df(X: pd.DataFrame, cm: dict, group_rank: dict) -> pd.DataFrame:
    """Таблица-инфо по фичам: группа, описание, dtype (с сортировкой по группам)."""
    rows = []
    for col in X.columns:
        group = cm_group(col, default="(без группы)")
        rows.append({
            "group_ru": group,
            "group_rank": group_rank.get(group, 999),
            "feature": col,
            "description_ru": cm_label(col),
            "dtype": str(X[col].dtype),
        })

    info_df = pd.DataFrame(rows)
    if info_df.empty:
        return info_df

    return (
        info_df.sort_values(["group_rank", "group_ru", "feature"])
        .drop(columns=["group_rank"])
        .reset_index(drop=True)
    )


print('Набор #1: с антропометрией')
X_with_body_info_df = make_features_info_df(X_with_body, cm, group_rank)
display(X_with_body_info_df)

print('Набор #2: без антропометрии')
X_no_body_info_df = make_features_info_df(X_no_body, cm, group_rank)
display(X_no_body_info_df)


Набор #1: с антропометрией


,group_ru,feature,description_ru,dtype
0,Признаки пищевых привычек,CAEC,Перекусы между приемами пищи,category
1,Признаки пищевых привычек,CALC,Употребление алкоголя,category
2,Признаки пищевых привычек,CH2O,Потребление воды в день,category
3,Признаки пищевых привычек,FAVC,Частое употребление высококалорийной пищи,category
4,Признаки пищевых привычек,FCVC,Частота употребления овощей,category
5,Признаки пищевых привычек,NCP,Количество основных приемов пищи,category
6,Признаки физического состояния,FAF,Частота физической активности,category
7,Признаки физического состояния,MTRANS,Используемый транспорт,category
8,Признаки физического состояния,SCC,Самоконтроль потребления калорий,category
9,Признаки физического состояния,TUE,Время использования электронных устройств,category


Набор #2: без антропометрии


,group_ru,feature,description_ru,dtype
0,Признаки пищевых привычек,CAEC,Перекусы между приемами пищи,category
1,Признаки пищевых привычек,CALC,Употребление алкоголя,category
2,Признаки пищевых привычек,CH2O,Потребление воды в день,category
3,Признаки пищевых привычек,FAVC,Частое употребление высококалорийной пищи,category
4,Признаки пищевых привычек,FCVC,Частота употребления овощей,category
5,Признаки пищевых привычек,NCP,Количество основных приемов пищи,category
6,Признаки физического состояния,FAF,Частота физической активности,category
7,Признаки физического состояния,MTRANS,Используемый транспорт,category
8,Признаки физического состояния,SCC,Самоконтроль потребления калорий,category
9,Признаки физического состояния,TUE,Время использования электронных устройств,category
